This notebooks takes the pickle file of filtered VCF files (SNPS) and reduces the feature space with a Random Forest algorithm.

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
# from keras.models import Sequential
# from keras.layers import Dense,Dropout, Flatten,BatchNormalization, GaussianNoise
import matplotlib.pyplot as plt
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
# from tensorflow.keras import initializers
# from tensorflow.keras import regularizers
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import os

In [2]:
os.chdir("/users/anair27/data/anair27")

Import the outcome variable from the clinical spreadsheet.

In [3]:
clinical = pd.read_csv("./singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_clinical/processed_clinical.csv")

In [4]:
diagnosis = clinical[["vital_status_Dead", "case_id"]]
diagnosis.head(10)

,vital_status_Dead,case_id
0,0,TCGA-78-8640
1,0,TCGA-05-5425
2,0,TCGA-55-7815
3,1,TCGA-44-7661
4,0,TCGA-97-7554
5,1,TCGA-J2-A4AD
6,1,TCGA-55-6981
7,0,TCGA-86-8281
8,1,TCGA-44-A4SU
9,1,TCGA-93-A4JO


In [5]:
def read_methylation(filepath, case_id):
#     arr = []
#     with open(filepath) as f:
#         lines = f.readlines()
#         for l in lines:
#             arr.append(l.split())
    return pd.read_csv(filepath, sep='\t', header = None).set_index(0).rename(columns={1:case_id})
    # return pd.DataFrame(arr)
            #     matrix = pd.DataFrame(arr).replace('NA', -1)
#     matrix = matrix.rename(columns={0: 'GENOMIC_REGION', 1: 'BETA_VALUE'})
#     matrix = matrix.set_index('GENOMIC_REGION').astype('float').transpose()
#     return matrix.rename(index = {'BETA_VALUE': case_id})

In [6]:
cases = os.listdir("data_by_cases")
cases[0:10]

['TCGA-35-4122',
 'TCGA-75-6203',
 'TCGA-75-5146',
 'TCGA-78-8648',
 'TCGA-55-A4DG',
 'TCGA-MP-A4SY',
 'TCGA-67-3771',
 'TCGA-44-A479',
 'TCGA-78-7156',
 'TCGA-55-7724']

In [7]:
epigenomic_data = []
l = len(cases)
for i, case in enumerate(cases):
    print(f"Case {i}/{l}")
    contents_gene_meth = os.listdir(os.path.join("data_by_cases", case, "dna_methylation"))
    if len(contents_gene_meth) == 0:
        print(f"{case} has no methylation data")
    else:
        filename = contents_gene_meth[0]
        path = os.path.join("data_by_cases", case, "dna_methylation", filename)
        epigenomic_data.append(read_methylation(path, case))
        # gene_exp_data.append(read_methylation(path, case))
# pd.concat(gene_exp_data, axis = 0)        

Case 0/521
Case 1/521
Case 2/521
Case 3/521
Case 4/521
Case 5/521
Case 6/521
Case 7/521
Case 8/521
Case 9/521
Case 10/521
Case 11/521
Case 12/521
Case 13/521
Case 14/521
Case 15/521
Case 16/521
Case 17/521
Case 18/521
Case 19/521
Case 20/521
Case 21/521
Case 22/521
Case 23/521
Case 24/521
Case 25/521
Case 26/521
Case 27/521
Case 28/521
Case 29/521
Case 30/521
Case 31/521
Case 32/521
Case 33/521
Case 34/521
TCGA-05-4245 has no methylation data
Case 35/521
Case 36/521
Case 37/521
Case 38/521
Case 39/521
Case 40/521
Case 41/521
Case 42/521
Case 43/521
Case 44/521
Case 45/521
Case 46/521
Case 47/521
Case 48/521
Case 49/521
Case 50/521
Case 51/521
Case 52/521
Case 53/521
Case 54/521
Case 55/521
Case 56/521
Case 57/521
Case 58/521
Case 59/521
TCGA-44-2664 has no methylation data
Case 60/521
Case 61/521
Case 62/521
Case 63/521
Case 64/521
Case 65/521
Case 66/521
Case 67/521
Case 68/521
Case 69/521
Case 70/521
Case 71/521
Case 72/521
Case 73/521
Case 74/521
Case 75/521
Case 76/521
Case 77/521


In [8]:
epigenomic_data[0]

,TCGA-35-4122
0,
cg22501393,0.036237
cg18895155,0.017973
cg27126442,0.503393
cg15264255,0.128218
cg18464559,0.023375
...,...
cg10265786,0.009094
cg08096038,0.864527
cg05535113,0.204837


In [9]:
len(epigenomic_data)

517

In [10]:
all_epigenomic = pd.concat(epigenomic_data, axis = 1)
all_epigenomic

,TCGA-35-4122,TCGA-75-6203,TCGA-75-5146,TCGA-78-8648,TCGA-55-A4DG,TCGA-MP-A4SY,TCGA-67-3771,TCGA-44-A479,TCGA-78-7156,TCGA-55-7724,...,TCGA-55-8090,TCGA-73-4670,TCGA-73-4668,TCGA-05-4422,TCGA-86-7714,TCGA-64-5775,TCGA-05-4418,TCGA-62-8398,TCGA-55-8097,TCGA-S2-AA1A
0,,,,,,,,,,,,,,,,,,,,,
cg22501393,0.036237,0.072509,0.082956,0.066536,0.087807,0.084434,0.032928,0.066551,0.072734,0.085435,...,0.073007,0.042333,0.032782,0.028549,0.082535,0.082374,0.034011,0.096798,0.072706,0.078151
cg18895155,0.017973,0.016707,0.013216,0.020478,0.025055,0.027104,0.020942,0.018722,0.016225,0.016588,...,0.016800,0.014032,0.017088,0.017525,0.022279,0.015395,0.020096,0.017866,0.018051,0.020128
cg27126442,0.503393,0.572085,0.614120,0.806440,0.463099,0.504607,0.353830,0.743840,0.231782,0.610359,...,0.341039,0.228794,0.468894,0.273344,0.341466,0.766534,0.312059,0.445671,0.385491,0.635967
cg15264255,0.128218,0.053483,0.053742,0.046997,0.054819,0.056879,0.100819,0.049452,0.048707,0.053750,...,0.056473,0.113974,0.129406,0.092835,0.053205,0.035275,0.153293,0.046682,0.038005,0.053797
cg18464559,0.023375,0.024562,0.026393,0.026298,0.031556,0.026780,0.022435,0.023429,0.031727,0.027241,...,0.025930,0.019299,0.016195,0.019415,0.033429,0.023159,0.017379,0.030446,0.030631,0.032406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg03930849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg17939569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg13365400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
all_epigenomic = all_epigenomic.dropna()
all_epigenomic

,TCGA-35-4122,TCGA-75-6203,TCGA-75-5146,TCGA-78-8648,TCGA-55-A4DG,TCGA-MP-A4SY,TCGA-67-3771,TCGA-44-A479,TCGA-78-7156,TCGA-55-7724,...,TCGA-55-8090,TCGA-73-4670,TCGA-73-4668,TCGA-05-4422,TCGA-86-7714,TCGA-64-5775,TCGA-05-4418,TCGA-62-8398,TCGA-55-8097,TCGA-S2-AA1A
0,,,,,,,,,,,,,,,,,,,,,
cg22501393,0.036237,0.072509,0.082956,0.066536,0.087807,0.084434,0.032928,0.066551,0.072734,0.085435,...,0.073007,0.042333,0.032782,0.028549,0.082535,0.082374,0.034011,0.096798,0.072706,0.078151
cg18895155,0.017973,0.016707,0.013216,0.020478,0.025055,0.027104,0.020942,0.018722,0.016225,0.016588,...,0.016800,0.014032,0.017088,0.017525,0.022279,0.015395,0.020096,0.017866,0.018051,0.020128
cg27126442,0.503393,0.572085,0.614120,0.806440,0.463099,0.504607,0.353830,0.743840,0.231782,0.610359,...,0.341039,0.228794,0.468894,0.273344,0.341466,0.766534,0.312059,0.445671,0.385491,0.635967
cg15264255,0.128218,0.053483,0.053742,0.046997,0.054819,0.056879,0.100819,0.049452,0.048707,0.053750,...,0.056473,0.113974,0.129406,0.092835,0.053205,0.035275,0.153293,0.046682,0.038005,0.053797
cg18464559,0.023375,0.024562,0.026393,0.026298,0.031556,0.026780,0.022435,0.023429,0.031727,0.027241,...,0.025930,0.019299,0.016195,0.019415,0.033429,0.023159,0.017379,0.030446,0.030631,0.032406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg23207527,0.834547,0.760980,0.770362,0.705605,0.810681,0.791004,0.839995,0.670632,0.875312,0.820711,...,0.887615,0.874821,0.845008,0.865579,0.955432,0.921748,0.863683,0.845899,0.847985,0.769892
cg20880234,0.021432,0.043046,0.038415,0.043201,0.056493,0.048977,0.021158,0.055979,0.042082,0.041723,...,0.042008,0.015630,0.021477,0.026571,0.048957,0.043156,0.021754,0.043807,0.040767,0.043275
cg10265786,0.009094,0.089766,0.038355,0.057632,0.062269,0.065119,0.010356,0.073144,0.065693,0.049020,...,0.059658,0.637641,0.013241,0.009065,0.056260,0.055920,0.008900,0.074081,0.056788,0.051019


In [12]:
all_epigenomic = all_epigenomic.transpose()

In [18]:
all_epigenomic = all_epigenomic.reset_index().rename(columns={"index": "case_id"})

In [22]:
all_epigenomic.to_csv("/users/anair27/data/anair27/singh-lab-TCGA-project/MADDi_model/data_processed/epigenomic_data.csv")

In [57]:
all_epigenomic.to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/all_epigenomic.pkl")

---


In [58]:
clinical = pd.read_csv("./singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_clinical/processed_clinical.csv")

In [59]:
diagnosis = clinical[["vital_status_Dead", "case_id"]]
diagnosis.head(10)

,vital_status_Dead,case_id
0,0,TCGA-78-8640
1,0,TCGA-05-5425
2,0,TCGA-55-7815
3,1,TCGA-44-7661
4,0,TCGA-97-7554
5,1,TCGA-J2-A4AD
6,1,TCGA-55-6981
7,0,TCGA-86-8281
8,1,TCGA-44-A4SU
9,1,TCGA-93-A4JO


In [66]:
all_epigenomic = pd.read_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/all_epigenomic.pkl")
all_epigenomic = all_epigenomic.reset_index().rename(columns={'index':'case_id'})


In [67]:
epigenomic_merge = all_epigenomic.merge(diagnosis, on = "case_id").drop_duplicates()

In [27]:
# gene_exp_merge.to_csv("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/all_gene_exp_FPKM.csv")

In [68]:
epigenomic_merge.head(10)

,case_id,cg22501393,cg18895155,cg27126442,cg15264255,cg18464559,cg20379125,cg12790134,cg07697569,cg13613532,...,cg14474880,cg11500797,cg04995095,cg19815376,cg23207527,cg20880234,cg10265786,cg26848248,cg09906309,vital_status_Dead
0,TCGA-35-4122,0.036237,0.017973,0.503393,0.128218,0.023375,0.025185,0.012751,0.018539,0.031869,...,0.049937,0.315507,0.377128,0.454613,0.834547,0.021432,0.009094,0.022390,0.059245,0
1,TCGA-75-6203,0.072509,0.016707,0.572085,0.053483,0.024562,0.040282,0.055873,0.016879,0.045806,...,0.051249,0.262323,0.744453,0.406976,0.760980,0.043046,0.089766,0.065425,0.036929,0
2,TCGA-75-5146,0.082956,0.013216,0.614120,0.053742,0.026393,0.062964,0.066770,0.013661,0.041391,...,0.045764,0.709449,0.369011,0.372617,0.770362,0.038415,0.038355,0.045358,0.028891,0
3,TCGA-78-8648,0.066536,0.020478,0.806440,0.046997,0.026298,0.046908,0.036235,0.018256,0.045744,...,0.071553,0.228389,0.686497,0.408400,0.705605,0.043201,0.057632,0.037235,0.040959,1
4,TCGA-55-A4DG,0.087807,0.025055,0.463099,0.054819,0.031556,0.075073,0.049878,0.021182,0.047862,...,0.048047,0.636668,0.423101,0.576572,0.810681,0.056493,0.062269,0.046892,0.045805,0
5,TCGA-MP-A4SY,0.084434,0.027104,0.504607,0.056879,0.026780,0.036102,0.042616,0.018484,0.044797,...,0.068959,0.470080,0.714256,0.491234,0.791004,0.048977,0.065119,0.045725,0.039118,1
6,TCGA-67-3771,0.032928,0.020942,0.353830,0.100819,0.022435,0.016335,0.013617,0.019215,0.019148,...,0.042046,0.171405,0.239074,0.611294,0.839995,0.021158,0.010356,0.020808,0.036040,0
7,TCGA-44-A479,0.066551,0.018722,0.743840,0.049452,0.023429,0.084409,0.052110,0.014370,0.057847,...,0.055291,0.248331,0.499029,0.526249,0.670632,0.055979,0.073144,0.049140,0.055190,0
8,TCGA-78-7156,0.072734,0.016225,0.231782,0.048707,0.031727,0.034550,0.073621,0.018055,0.037837,...,0.055886,0.125026,0.231479,0.195041,0.875312,0.042082,0.065693,0.064725,0.028224,1
9,TCGA-55-7724,0.085435,0.016588,0.610359,0.053750,0.027241,0.062259,0.065290,0.017147,0.039874,...,0.044478,0.335961,0.612031,0.378562,0.820711,0.041723,0.049020,0.058300,0.055921,0


In [69]:
cols = list(set(epigenomic_merge.columns) - set(["vital_status_Dead", "case_id"]))
# X = pd.DataFrame(gene_exp_merge[cols].values.astype(float))
# y = gene_exp_merge["vital_status_Dead"].astype(int).values
X = epigenomic_merge[cols].astype(float)
y = epigenomic_merge["vital_status_Dead"].astype(int)

In [70]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(X, y)
print('Original dataset shape', Counter(y))
print('Resample dataset shape', Counter(y_ros))

Original dataset shape Counter({0: 329, 1: 188})
Resample dataset shape Counter({0: 329, 1: 329})


In [71]:
X_train, X_test, y_train, y_test = train_test_split(x_ros, y_ros, test_size=0.1)

In [72]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [73]:
(sel.get_support())

array([False, False, False, ...,  True, False, False])

In [74]:
X_train

,cg24242194,cg11220635,cg20430816,cg16794682,cg11665562,cg04329382,cg05953394,cg04733989,cg07705908,cg27653134,...,cg21509097,cg16907514,cg07715387,cg01405761,cg11521325,cg17812013,cg11191210,cg19853494,cg09391966,cg11388238
271,0.036028,0.059712,0.082158,0.893986,0.040241,0.359123,0.032681,0.016723,0.426302,0.545590,...,0.089025,0.079586,0.062637,0.102976,0.594855,0.071981,0.248617,0.691671,0.041092,0.115801
182,0.029188,0.067186,0.139978,0.913577,0.065872,0.458872,0.028975,0.013985,0.462643,0.692013,...,0.099413,0.193787,0.068391,0.137735,0.494621,0.060942,0.126688,0.515074,0.043167,0.072950
497,0.034709,0.046772,0.126774,0.933441,0.077575,0.337871,0.034672,0.011150,0.363597,0.824569,...,0.134749,0.179974,0.066074,0.419020,0.531967,0.045970,0.285155,0.686957,0.031751,0.058597
78,0.035681,0.058520,0.051735,0.925960,0.063799,0.421913,0.028699,0.013066,0.534988,0.802333,...,0.083481,0.204749,0.069246,0.118015,0.507339,0.046223,0.095300,0.425603,0.031257,0.073084
91,0.013063,0.080993,0.064425,0.866313,0.019183,0.219019,0.023683,0.010041,0.538487,0.771074,...,0.059114,0.061839,0.013255,0.053824,0.418192,0.016975,0.040648,0.209911,0.011272,0.020227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,0.024806,0.053208,0.134421,0.879649,0.053853,0.662895,0.025122,0.012578,0.515656,0.837062,...,0.082369,0.164938,0.058257,0.383315,0.444597,0.069702,0.156623,0.302600,0.028622,0.048517
71,0.032401,0.053911,0.115461,0.773873,0.059116,0.389839,0.018770,0.013168,0.439684,0.723410,...,0.088641,0.119940,0.054367,0.216939,0.408173,0.056104,0.090886,0.275505,0.036735,0.039212
137,0.055820,0.050888,0.142588,0.956078,0.110763,0.348816,0.187894,0.013173,0.497730,0.738658,...,0.099797,0.252626,0.081745,0.151375,0.600554,0.039902,0.428956,0.324492,0.044830,0.055258
100,0.033916,0.051952,0.082118,0.927572,0.058985,0.663358,0.034713,0.011950,0.553719,0.783783,...,0.110188,0.067564,0.062300,0.105395,0.382547,0.048946,0.129523,0.595140,0.032554,0.053201


In [75]:
selected_feat= pd.DataFrame(X_train).columns[(sel.get_support())]
len(selected_feat)

3971

In [76]:
# feature reduction by RF classifier
selected_feat


Index(['cg04329382', 'cg05953394', 'cg04733989', 'cg23186643', 'cg19091641',
       'cg09537031', 'cg21137706', 'cg05749577', 'cg26798786', 'cg04765422',
       ...
       'cg02100848', 'cg06609475', 'cg21449843', 'cg08707078', 'cg23855566',
       'cg09599653', 'cg08680085', 'cg21509097', 'cg11191210', 'cg19853494'],
      dtype='object', length=3971)

In [78]:
l = ["case_id", "vital_status_Dead"]
l.extend(selected_feat)

In [79]:
#saving the features with the old dataframe so that the overlap test set can still be used when combining all data
epigenomic_merge[l].to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/all_epigenomic_select.pkl")

In [80]:
X_train[selected_feat].to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/epigenomic_X_train.pkl")
y_train.to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/epigenomic_y_train.pkl")

X_test[selected_feat].to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/epigenomic_X_test.pkl")
y_test.to_pickle("singh-lab-TCGA-project/MADDi_model/MADDi/preprocess_genetic/epigenomic_y_test.pkl")

---

In [ ]:
#reading all the SNP files
vcf = pd.read_pickle("all_vcfs.pkl")

In [ ]:
#reading in the diagnosis data
m = pd.read_csv("diagnosis_full.csv").drop("Unnamed: 0", axis=1).rename(columns = {"Subject":"subject", "GROUP": "label"})

In [ ]:
#making sure all the diagnosis agree
m = m[m["label"] != -1]

In [ ]:
#merging SNPs with diagnosis
vcf = vcf.merge(m[["subject", "label"]], on = "subject")

In [ ]:
vcf = vcf.drop_duplicates()

In [ ]:
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv")

#removing ids from the overlap test set, saving it as a new variable
vcf1 = vcf[~vcf["subject"].isin(list(ts["subject"].values))]

In [ ]:
# Using Random Forest to reduce feature dimensions
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))

In [ ]:
cols = list(set(vcf1.columns) - set(["subject", "Group", "label"]))
X = vcf1[cols].values.astype(int)
y = vcf1["label"].astype(int).values

for i in range(len(y)):
    y[i] = y[i]-1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [ ]:
#fitting random forest only on the training data so that there is not influence on the testing performance
sel.fit(X_train, y_train)

In [ ]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

In [ ]:
print(selected_feat)

In [ ]:
l = ["label", "subject", "Group"]
l.extend(selected_feat)

In [ ]:
#saving the features with the old dataframe so that the overlap test set can still be used when combining all data
vcf[l].to_pickle("vcf_select.pkl")

In [ ]:
X_train[selected_feat].to_pickle("X_train_vcf.pkl")
y_train.to_pickle("y_train_vcf.pkl")

X_test[selected_feat].to_pickle("X_test_vcf.pkl")
y_test.to_pickle("y_test_vcf.pkl")